## Get facial landmarks using pretrained dlib


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from keras import models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AveragePooling2D
# from keras.optimizers import RMSProp,Adam
from tensorflow.keras.utils import to_categorical
from keras.layers import BatchNormalization
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, LSTM, merge
# from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
import csv
from PIL import Image    
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tqdm import tqdm
import numpy as np # linear algebra
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
import collections
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dropout

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# path = '/content/drive/My Drive/'
path = '/content/drive/MyDrive/'


os.listdir(path)

In [ ]:
data = pd.read_csv(path+'challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv')

In [ ]:
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label
# use pandarallel
def plot_examples(label=0):
    fig, axs = plt.subplots(1, 5, figsize=(25, 12))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    for i in range(5):
        idx = data[data['emotion']==label].index[i]
        axs[i].imshow(train_images[idx][:,:,0], cmap='gray')
        axs[i].set_title(emotions[train_labels[idx].argmax()])
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
 
def plot_all_emotions():
    fig, axs = plt.subplots(1, 7, figsize=(30, 12))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    for i in range(7):
        idx = data[data['emotion']==i].index[i]
        axs[i].imshow(train_images[idx][:,:,0], cmap='gray')
        axs[i].set_title(emotions[train_labels[idx].argmax()])
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        
def plot_image_and_emotion(test_image_array, test_image_label, pred_test_labels, image_number):
    """ Function to plot the image and compare the prediction results with the label """
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=False)
    
    bar_label = emotions.values()
    
    axs[0].imshow(test_image_array[image_number], 'gray')
    axs[0].set_title(emotions[test_image_label[image_number]])
    
    axs[1].bar(bar_label, pred_test_labels[image_number], color='orange', alpha=0.7)
    axs[1].grid()
    
    plt.show()
 
def plot_compare_distributions(array1, array2, title1='', title2=''):
    df_array1 = pd.DataFrame()
    df_array2 = pd.DataFrame()
    df_array1['emotion'] = array1.argmax(axis=1)
    df_array2['emotion'] = array2.argmax(axis=1)
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=False)
    x = emotions.values()
    
    y = df_array1['emotion'].value_counts()
    keys_missed = list(set(emotions.keys()).difference(set(y.keys())))
    for key_missed in keys_missed:
        y[key_missed] = 0
    axs[0].bar(x, y.sort_index(), color='orange')
    axs[0].set_title(title1)
    axs[0].grid()
    
    y = df_array2['emotion'].value_counts()
    keys_missed = list(set(emotions.keys()).difference(set(y.keys())))
    for key_missed in keys_missed:
        y[key_missed] = 0
    axs[1].bar(x, y.sort_index())
    axs[1].set_title(title2)
    axs[1].grid()
    
    plt.show()

In [ ]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
train_image_array, train_image_label = prepare_data(data[data[' Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data[' Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(data[data[' Usage']=='PublicTest'])

In [ ]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
# train_images = train_images.astype('float32')/255
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
# val_images = val_images.astype('float32')/255
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
# test_images = test_images.astype('float32')/255

In [ ]:
train_labels = to_categorical(train_image_label)
val_labels = to_categorical(val_image_label)
test_labels = to_categorical(test_image_label)

print(np.shape(train_labels))

(28709, 7)


In [ ]:
import cv2
import numpy as np
import dlib

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/My Drive/shape_predictor_68_face_landmarks.dat")

In [ ]:
train_final=[]
detected=[]
for q in range(28709):
  
  gray=np.array(train_images[q][:,:,:],dtype=np.uint8)
  gray=cv2.resize(gray,(48,48))
  faces=detector(gray,1)

  if (np.shape(faces)[0]==0):
    
    continue
  detected.append(q)
  for face in faces:
    x1=face.left()
    y1=face.top()
    x2=face.right()
    y2=face.bottom()
 
    landmarks=predictor(gray, face)
    xlist=[]
    ylist=[]
    for n in range(0,68):
      xlist.append(float(landmarks.part(n).x))
      ylist.append(float(landmarks.part(n).y))

    xmean=np.mean(xlist)
    ymean=np.mean(ylist)
    xcentral=[(x-xmean) for x in xlist] 
    ycentral=[(y-ymean) for y in ylist]
    train_landmark=[]

    for i,j in zip(xcentral,ycentral):

      train_landmark.append([[i],[j]])

    train_final.append(train_landmark)
  


In [ ]:
train_final, final_train_labels = train_final, train_labels[detected]

In [ ]:
val_final=[]
detected=[]
for q in range(3589):
 
  gray=np.array(val_images[q][:,:,:],dtype=np.uint8)
  gray=cv2.resize(gray,(48,48))
  faces=detector(gray,1)

  if (np.shape(faces)[0]==0):
    
    continue
  detected.append(q)
  for face in faces:
    x1=face.left()
    y1=face.top()
    x2=face.right()
    y2=face.bottom()
 
    landmarks=predictor(gray, face)
    xlist=[]
    ylist=[]
    for n in range(0,68):
      xlist.append(float(landmarks.part(n).x))
      ylist.append(float(landmarks.part(n).y))

    xmean=np.mean(xlist)
    ymean=np.mean(ylist)
    xcentral=[(x-xmean) for x in xlist] 
    ycentral=[(y-ymean) for y in ylist]
    train_landmark=[]

    for i,j in zip(xcentral,ycentral):

      train_landmark.append([[i],[j]])

    val_final.append(train_landmark)
  

In [ ]:
final_val_images, final_val_labels = val_images[detected], val_labels[detected]

In [ ]:
test_final=[]
detected=[]
for q in range(3589):
 
  gray=np.array(test_images[q][:,:,:],dtype=np.uint8)
  gray=cv2.resize(gray,(48,48))
  faces=detector(gray,1)

  if (np.shape(faces)[0]==0):
    
    continue
  detected.append(q)
  for face in faces:
    x1=face.left()
    y1=face.top()
    x2=face.right()
    y2=face.bottom()
 
    landmarks=predictor(gray, face)
    xlist=[]
    ylist=[]
    for n in range(0,68):
      xlist.append(float(landmarks.part(n).x))
      ylist.append(float(landmarks.part(n).y))

    xmean=np.mean(xlist)
    ymean=np.mean(ylist)
    xcentral=[(x-xmean) for x in xlist] 
    ycentral=[(y-ymean) for y in ylist]
    train_landmark=[]

    for i,j in zip(xcentral,ycentral):

      train_landmark.append([[i],[j]])

    test_final.append(train_landmark)
  

In [ ]:
final_test_images, final_test_lables = test_images[detected], test_labels[detected]

In [ ]:
from keras.layers import Dense, PReLU, LeakyReLU
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras import optimizers
from keras.regularizers import l2

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_final=np.array(train_final)
final_train_labels=np.array(final_train_labels)
val_final=np.array(val_final)
final_val_labels=np.array(final_val_labels)
final_test_images = np.array(final_test_images)
final_test_lables = np.array(final_test_lables)
test_final=np.array(test_final)


print(np.shape(train_final))
print(np.shape(final_train_labels))
print(np.shape(val_final))
print(np.shape(final_val_labels))
print(np.shape(test_final))
print(np.shape(final_test_lables))

(19984, 68, 2, 1)
(19984, 7)
(2471, 68, 2, 1)
(2471, 7)
(2487, 68, 2, 1)
(2487, 7)


## Using SVM to predict the emotion on the basis of the relative positions of the landmarks.

In [ ]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(train_final, final_train_labels)
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(train_final, final_train_labels)
poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(train_final, final_train_labels)
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(train_final, final_train_labels)

In [ ]:
linear_pred = linear.predict(test_final)
poly_pred = poly.predict(test_final)
rbf_pred = rbf.predict(test_final)
sig_pred = sig.predict(test_final)

In [ ]:
accuracy_lin = linear.score(test_final, final_test_labels)
accuracy_poly = poly.score(test_final, final_test_labels)
accuracy_rbf = rbf.score(test_final, final_test_labels)
accuracy_sig = sig.score(test_final, final_test_labels)
print(“Accuracy Linear Kernel:”, accuracy_lin)
print(“Accuracy Polynomial Kernel:”, accuracy_poly)
print(“Accuracy Radial Basis Kernel:”, accuracy_rbf)
print(“Accuracy Sigmoid Kernel:”, accuracy_sig